In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the dataset
data = pd.read_csv("drowsiness_data.csv")

# Split the dataset into features (X) and labels (y)
X = data[["EAR", "MAR"]].values
y = data["Label"].values

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data (optional, but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the Keras model
model = Sequential()
model.add(Dense(32, input_dim=2, activation="relu"))  # 2 inputs: EAR and MAR
model.add(Dense(16, activation="relu"))
model.add(Dense(1, activation="sigmoid"))  # Binary classification: Drowsy (1) or Not Drowsy (0)

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the trained model
model.save("drowsiness_detection_model.keras")


Epoch 1/50


c:\Users\Tarun\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8881 - loss: 0.5599 - val_accuracy: 0.9457 - val_loss: 0.2993
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9329 - loss: 0.2436 - val_accuracy: 0.9565 - val_loss: 0.1634
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9356 - loss: 0.1678 - val_accuracy: 0.9620 - val_loss: 0.1497
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9321 - loss: 0.1710 - val_accuracy: 0.9620 - val_loss: 0.1467
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9427 - loss: 0.1534 - val_accuracy: 0.9620 - val_loss: 0.1450
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9349 - loss: 0.1496 - val_accuracy: 0.9620 - val_loss: 0.1460
Epoch 7/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9534 - loss: 0.1259 - val_accuracy: 0.9620 - val_loss: 0.1455
Epoch 8/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9418 - loss: 0.1428 - val_accuracy: 0.9620 - val_loss: 0.1455
Epo

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import csv
import time

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

# Landmark indices for eyes and mouth
LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]
MOUTH_IDX = [61, 81, 78, 13, 312, 308, 291, 402, 318, 324, 14, 88]

# Function to calculate Eye Aspect Ratio (EAR)
def EAR(eye_landmarks):
    point1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    point2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    distance = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    ear_aspect_ratio = (point1 + point2) / (2.0 * distance)
    return ear_aspect_ratio

# Function to calculate Mouth Aspect Ratio (MAR)
def MAR(mouth_landmarks):
    point = np.linalg.norm(mouth_landmarks[0] - mouth_landmarks[6])
    point1 = np.linalg.norm(mouth_landmarks[2] - mouth_landmarks[10])
    point2 = np.linalg.norm(mouth_landmarks[4] - mouth_landmarks[8])
    Ypoint = (point1 + point2) / 2.0
    mouth_aspect_ratio = Ypoint / point
    return mouth_aspect_ratio

# Open webcam and collect EAR, MAR with labels (0: Not Drowsy, 1: Drowsy)
def collect_data():
    webcamera = cv2.VideoCapture(0)
    data = []
    
    print("Press 'q' to quit.")
    print("Press 'd' when you feel drowsy and 'n' when you are not drowsy to label the data.")
    
    while True:
        ret, frame = webcamera.read()
        frame = cv2.flip(frame, 1)  # Flip the frame horizontally for mirror view
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb_frame)

        if result.multi_face_landmarks:
            for face_landmarks in result.multi_face_landmarks:
                landmarks = np.array([[p.x, p.y] for p in face_landmarks.landmark])

                # Extract left and right eye landmarks
                left_eye = landmarks[LEFT_EYE_IDX]
                right_eye = landmarks[RIGHT_EYE_IDX]
                
                # Calculate EAR for eyes
                leftEAR = EAR(left_eye)
                rightEAR = EAR(right_eye)
                ear = (leftEAR + rightEAR) / 2.0
                
                # Extract mouth landmarks and calculate MAR
                mouth = landmarks[MOUTH_IDX]
                mouEAR = MAR(mouth)

                # Show EAR and MAR on the frame
                cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"MAR: {mouEAR:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Check for key presses once per frame
        key = cv2.waitKey(1) & 0xFF
        if key == ord("d"):
            label = 1  # Drowsy
            print(f"Collected data: EAR={ear}, MAR={mouEAR}, Label=Drowsy")
            data.append([ear, mouEAR, label])  # Collect EAR, MAR, and label
        elif key == ord("n"):
            label = 0  # Not Drowsy
            print(f"Collected data: EAR={ear}, MAR={mouEAR}, Label=Not Drowsy")
            data.append([ear, mouEAR, label])  # Collect EAR, MAR, and label
        elif key == ord("q"):
            break

        cv2.imshow("Frame", frame)

    webcamera.release()
    cv2.destroyAllWindows()

    # Check if data has been collected before writing to CSV
    if data:
        print(f"Saving {len(data)} rows of data to drowsiness_data.csv")
        # Save collected data to a CSV file
        with open("daata.csv", "w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["EAR", "MAR", "Label"])  # Column names
            writer.writerows(data)
    else:
        print("No data collected!")

collect_data()

Press 'q' to quit.
Press 'd' when you feel drowsy and 'n' when you are not drowsy to label the data.
No data collected!


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the dataset
data = pd.read_csv("C:\\Users\\Tarun\\OneDrive\\Desktop\\New folder\\drowsiness_data_.csv")

# Split the dataset into features (X) and labels (y)
X = data[["EAR", "MAR"]].values
y = data["Label"].values

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data (optional, but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the Keras model
model = Sequential()
model.add(Dense(32, input_dim=2, activation="relu"))  # 2 inputs: EAR and MAR
model.add(Dense(16, activation="relu"))
model.add(Dense(1, activation="sigmoid"))  # Binary classification: Drowsy (1) or Not Drowsy (0)

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the trained model
model.save("drowsiness_detection_model1.keras")

c:\Users\Tarun\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6817 - loss: 0.6539 - val_accuracy: 0.9877 - val_loss: 0.4893
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9506 - loss: 0.4389 - val_accuracy: 1.0000 - val_loss: 0.2949
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9522 - loss: 0.2701 - val_accuracy: 1.0000 - val_loss: 0.1547
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9497 - loss: 0.1640 - val_accuracy: 1.0000 - val_loss: 0.0931
Epoch 5/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9643 - loss: 0.1102 - val_accuracy: 1.0000 - val_loss: 0.0639
Epoch 6/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9488 - loss: 0.1187 - val_accuracy: 1.0000 - val_loss: 0.0485
Epoch 7/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9620 - loss: 0.0889 - val_accuracy: 1.0000 - val_loss: 0.0422
Epoch 8/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9799 - loss: 0.0567 - val_accuracy: 1.0000 - val_loss:

In [5]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix

# Load the saved model
model = load_model("drowsiness_detection_model1.keras")

# Use the test set to evaluate the model
# X_test and y_test should be the test data from the training process
# (Ensure they're still available in the current session)

# Make predictions
y_pred = model.predict(X_test)

# Convert probabilities to binary predictions (threshold = 0.5)
y_pred_binary = (y_pred > 0.5).astype(int).flatten()

# Evaluate the model's performance
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_binary))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_binary))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Confusion Matrix:
[[46  0]
 [ 0 35]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        46
           1       1.00      1.00      1.00        35

    accuracy                           1.00        81
   macro avg       1.00      1.00      1.00        81
weighted avg       1.00      1.00      1.00        81



In [ ]:
#works
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import time
import threading
from playsound import playsound

# Load the trained model
model = load_model("drowsiness_detection_model1.keras")

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

# Landmark indices for eyes and mouth
LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]
MOUTH_IDX = [61, 81, 78, 13, 312, 308, 291, 402, 318, 324, 14, 88]

# Function to calculate Eye Aspect Ratio (EAR)
def EAR(eye_landmarks):
    point1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    point2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    distance = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    ear_aspect_ratio = (point1 + point2) / (2.0 * distance)
    return ear_aspect_ratio

# Function to calculate Mouth Aspect Ratio (MAR)
def MAR(mouth_landmarks):
    point = np.linalg.norm(mouth_landmarks[0] - mouth_landmarks[6])
    point1 = np.linalg.norm(mouth_landmarks[2] - mouth_landmarks[10])
    point2 = np.linalg.norm(mouth_landmarks[4] - mouth_landmarks[8])
    Ypoint = (point1 + point2) / 2.0
    mouth_aspect_ratio = Ypoint / point
    return mouth_aspect_ratio

# Function to play beep sound
def play_beep():
    playsound("beep.mp3")  # Replace with the path to your beep sound file

# Open webcam and predict drowsiness
def predict_drowsiness():
    webcamera = cv2.VideoCapture(0)
    
    print("Press 'q' to quit.")
    
    drowsy_start_time = None  # Track when drowsy state starts
    
    while True:
        ret, frame = webcamera.read()
        frame = cv2.flip(frame, 1)  # Flip the frame horizontally for mirror view
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb_frame)

        if result.multi_face_landmarks:
            for face_landmarks in result.multi_face_landmarks:
                landmarks = np.array([[p.x, p.y] for p in face_landmarks.landmark])

                # Extract left and right eye landmarks
                left_eye = landmarks[LEFT_EYE_IDX]
                right_eye = landmarks[RIGHT_EYE_IDX]
                
                # Calculate EAR for eyes
                leftEAR = EAR(left_eye)
                rightEAR = EAR(right_eye)
                ear = (leftEAR + rightEAR) / 2.0
                
                # Extract mouth landmarks and calculate MAR
                mouth = landmarks[MOUTH_IDX]
                mouEAR = MAR(mouth)

                # Prepare input for the model
                features = np.array([[ear, mouEAR]])  # Model expects 2D input
                prediction = model.predict(features)
                label = "Drowsy" if prediction[0][0] > 0.5 else "Not Drowsy"

                # Display the prediction and features
                cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"MAR: {mouEAR:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"Prediction: {label}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                # Handle drowsiness state persistence
                if label == "Drowsy":
                    if drowsy_start_time is None:
                        drowsy_start_time = time.time()
                    elif time.time() - drowsy_start_time > 0.5:  # If drowsy for >0.5 seconds
                        threading.Thread(target=play_beep).start()
                else:
                    drowsy_start_time = None  # Reset if not drowsy

        # Check for key press to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        cv2.imshow("Drowsiness Detection", frame)

    webcamera.release()
    cv2.destroyAllWindows()

# Run the prediction function
predict_drowsiness()




In [7]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import time
import threading
import os

# Load the trained model
model = load_model("drowsiness_detection_model1.keras")

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

# Landmark indices for eyes and mouth
LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]
MOUTH_IDX = [61, 81, 78, 13, 312, 308, 291, 402, 318, 324, 14, 88]

# Function to calculate Eye Aspect Ratio (EAR)
def EAR(eye_landmarks):
    point1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    point2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    distance = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    ear_aspect_ratio = (point1 + point2) / (2.0 * distance)
    return ear_aspect_ratio

# Function to calculate Mouth Aspect Ratio (MAR)
def MAR(mouth_landmarks):
    point = np.linalg.norm(mouth_landmarks[0] - mouth_landmarks[6])
    point1 = np.linalg.norm(mouth_landmarks[2] - mouth_landmarks[10])
    point2 = np.linalg.norm(mouth_landmarks[4] - mouth_landmarks[8])
    Ypoint = (point1 + point2) / 2.0
    mouth_aspect_ratio = Ypoint / point
    return mouth_aspect_ratio

# Function to play beep sound
def play_beep():
    os.system('echo -e "\a"')  # Beep sound (works on most systems)

# Open webcam and predict drowsiness
def predict_drowsiness():
    webcamera = cv2.VideoCapture(0)
    
    print("Press 'q' to quit.")
    
    drowsy_start_time = None  # Track when drowsy state starts
    
    while True:
        ret, frame = webcamera.read()
        frame = cv2.flip(frame, 1)  # Flip the frame horizontally for mirror view
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb_frame)

        if result.multi_face_landmarks:
            for face_landmarks in result.multi_face_landmarks:
                landmarks = np.array([[p.x, p.y] for p in face_landmarks.landmark])

                # Extract left and right eye landmarks
                left_eye = landmarks[LEFT_EYE_IDX]
                right_eye = landmarks[RIGHT_EYE_IDX]
                
                # Calculate EAR for eyes
                leftEAR = EAR(left_eye)
                rightEAR = EAR(right_eye)
                ear = (leftEAR + rightEAR) / 2.0
                
                # Extract mouth landmarks and calculate MAR
                mouth = landmarks[MOUTH_IDX]
                mouEAR = MAR(mouth)

                # Prepare input for the model
                features = np.array([[ear, mouEAR]])  # Model expects 2D input
                prediction = model.predict(features)
                label = "Drowsy" if prediction[0][0] > 0.5 else "Not Drowsy"

                # Display the prediction and features
                cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"MAR: {mouEAR:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"Prediction: {label}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                # Handle drowsiness state persistence
                if label == "Drowsy":
                    if drowsy_start_time is None:
                        drowsy_start_time = time.time()
                    elif time.time() - drowsy_start_time > 0.5:  # If drowsy for >0.5 seconds
                        threading.Thread(target=play_beep).start()
                else:
                    drowsy_start_time = None  # Reset if not drowsy

        # Check for key press to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        cv2.imshow("Drowsiness Detection", frame)

    webcamera.release()
    cv2.destroyAllWindows()

# Run the prediction function
predict_drowsiness()


Press 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms

In [8]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("drowsiness_detection_model1.keras")

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

# Landmark indices for eyes and mouth
LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]
MOUTH_IDX = [61, 81, 78, 13, 312, 308, 291, 402, 318, 324, 14, 88]

# Function to calculate Eye Aspect Ratio (EAR)
def EAR(eye_landmarks):
    point1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    point2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    distance = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    ear_aspect_ratio = (point1 + point2) / (2.0 * distance)
    return ear_aspect_ratio

# Function to calculate Mouth Aspect Ratio (MAR)
def MAR(mouth_landmarks):
    point = np.linalg.norm(mouth_landmarks[0] - mouth_landmarks[6])
    point1 = np.linalg.norm(mouth_landmarks[2] - mouth_landmarks[10])
    point2 = np.linalg.norm(mouth_landmarks[4] - mouth_landmarks[8])
    Ypoint = (point1 + point2) / 2.0
    mouth_aspect_ratio = Ypoint / point
    return mouth_aspect_ratio

# Open webcam and predict drowsiness
def predict_drowsiness():
    webcamera = cv2.VideoCapture(0)
    
    print("Press 'q' to quit.")
    
    while True:
        ret, frame = webcamera.read()
        frame = cv2.flip(frame, 1)  # Flip the frame horizontally for mirror view
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb_frame)

        if result.multi_face_landmarks:
            for face_landmarks in result.multi_face_landmarks:
                landmarks = np.array([[p.x, p.y] for p in face_landmarks.landmark])

                # Extract left and right eye landmarks
                left_eye = landmarks[LEFT_EYE_IDX]
                right_eye = landmarks[RIGHT_EYE_IDX]
                
                # Calculate EAR for eyes
                leftEAR = EAR(left_eye)
                rightEAR = EAR(right_eye)
                ear = (leftEAR + rightEAR) / 2.0
                
                # Extract mouth landmarks and calculate MAR
                mouth = landmarks[MOUTH_IDX]
                mouEAR = MAR(mouth)

                # Prepare input for the model
                features = np.array([[ear, mouEAR]])  # Model expects 2D input
                prediction = model.predict(features)
                label = "Drowsy" if prediction[0][0] > 0.5 else "Not Drowsy"

                # Display the prediction and features
                cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"MAR: {mouEAR:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"Prediction: {label}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Check for key press to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        cv2.imshow("Drowsiness Detection", frame)

    webcamera.release()
    cv2.destroyAllWindows()

# Run the prediction function
predict_drowsiness()


Press 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
from playsound import playsound
import threading

# Load the trained model
model = load_model("drowsiness_detection_model1.keras")

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

# Landmark indices for eyes and mouth
LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]
MOUTH_IDX = [61, 81, 78, 13, 312, 308, 291, 402, 318, 324, 14, 88]

# Function to calculate Eye Aspect Ratio (EAR)
def EAR(eye_landmarks):
    point1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    point2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    distance = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    ear_aspect_ratio = (point1 + point2) / (2.0 * distance)
    return ear_aspect_ratio

# Function to calculate Mouth Aspect Ratio (MAR)
def MAR(mouth_landmarks):
    point = np.linalg.norm(mouth_landmarks[0] - mouth_landmarks[6])
    point1 = np.linalg.norm(mouth_landmarks[2] - mouth_landmarks[10])
    point2 = np.linalg.norm(mouth_landmarks[4] - mouth_landmarks[8])
    Ypoint = (point1 + point2) / 2.0
    mouth_aspect_ratio = Ypoint / point
    return mouth_aspect_ratio

# Function to play an alert sound
def play_alert():
    playsound("beep.mp3")  # Replace with your desired alert sound file

# Open webcam and predict drowsiness
def predict_drowsiness():
    webcamera = cv2.VideoCapture(0)
    
    print("Press 'q' to quit.")
    
    drowsy_alert_playing = False  # Prevent overlapping sound threads
    
    while True:
        ret, frame = webcamera.read()
        frame = cv2.flip(frame, 1)  # Flip the frame horizontally for mirror view
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb_frame)

        if result.multi_face_landmarks:
            for face_landmarks in result.multi_face_landmarks:
                landmarks = np.array([[p.x, p.y] for p in face_landmarks.landmark])

                # Extract left and right eye landmarks
                left_eye = landmarks[LEFT_EYE_IDX]
                right_eye = landmarks[RIGHT_EYE_IDX]
                
                # Calculate EAR for eyes
                leftEAR = EAR(left_eye)
                rightEAR = EAR(right_eye)
                ear = (leftEAR + rightEAR) / 2.0
                
                # Extract mouth landmarks and calculate MAR
                mouth = landmarks[MOUTH_IDX]
                mouEAR = MAR(mouth)

                # Prepare input for the model
                features = np.array([[ear, mouEAR]])  # Model expects 2D input
                prediction = model.predict(features)
                label = "Drowsy" if prediction[0][0] > 0.5 else "Not Drowsy"

                # Display the prediction and features
                cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"MAR: {mouEAR:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"Prediction: {label}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                # Play alert sound if drowsy and not already playing
                if label == "Drowsy" and not drowsy_alert_playing:
                    drowsy_alert_playing = True
                    threading.Thread(target=play_alert).start()
                elif label == "Not Drowsy":
                    drowsy_alert_playing = False

        # Check for key press to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        cv2.imshow("Drowsiness Detection", frame)

    webcamera.release()
    cv2.destroyAllWindows()

# Run the prediction function
predict_drowsiness()


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import time
import threading
from playsound import playsound

# Load the trained model
model = load_model("drowsiness_detection_model1.keras")

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

# Landmark indices for eyes and mouth
LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]
MOUTH_IDX = [61, 81, 78, 13, 312, 308, 291, 402, 318, 324, 14, 88]

# Function to calculate Eye Aspect Ratio (EAR)
def EAR(eye_landmarks):
    point1 = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    point2 = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    distance = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    ear_aspect_ratio = (point1 + point2) / (2.0 * distance)
    return ear_aspect_ratio

# Function to calculate Mouth Aspect Ratio (MAR)
def MAR(mouth_landmarks):
    point = np.linalg.norm(mouth_landmarks[0] - mouth_landmarks[6])
    point1 = np.linalg.norm(mouth_landmarks[2] - mouth_landmarks[10])
    point2 = np.linalg.norm(mouth_landmarks[4] - mouth_landmarks[8])
    Ypoint = (point1 + point2) / 2.0
    mouth_aspect_ratio = Ypoint / point
    return mouth_aspect_ratio

# Function to play beep sound
def play_beep():
    playsound("beeep.mp3")  # Replace with the path to your beep sound file

# Open webcam and predict drowsiness
def predict_drowsiness():
    webcamera = cv2.VideoCapture(0)
    
    print("Press 'q' to quit.")
    
    drowsy_start_time = None  # Track when drowsy state starts
    
    while True:
        ret, frame = webcamera.read()
        frame = cv2.flip(frame, 1)  # Flip the frame horizontally for mirror view
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb_frame)

        if result.multi_face_landmarks:
            for face_landmarks in result.multi_face_landmarks:
                landmarks = np.array([[p.x, p.y] for p in face_landmarks.landmark])

                # Extract left and right eye landmarks
                left_eye = landmarks[LEFT_EYE_IDX]
                right_eye = landmarks[RIGHT_EYE_IDX]
                
                # Calculate EAR for eyes
                leftEAR = EAR(left_eye)
                rightEAR = EAR(right_eye)
                ear = (leftEAR + rightEAR) / 2.0
                
                # Extract mouth landmarks and calculate MAR
                mouth = landmarks[MOUTH_IDX]
                mouEAR = MAR(mouth)

                # Prepare input for the model
                features = np.array([[ear, mouEAR]])  # Model expects 2D input
                prediction = model.predict(features)
                label = "Drowsy" if prediction[0][0] > 0.5 else "Not Drowsy"

                # Display the prediction and features
                cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"MAR: {mouEAR:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"Prediction: {label}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                # Handle drowsiness state persistence
                if label == "Drowsy":
                    if drowsy_start_time is None:
                        drowsy_start_time = time.time()
                    elif time.time() - drowsy_start_time > 0.5:  # If drowsy for >0.5 seconds
                        threading.Thread(target=play_beep).start()
                else:
                    drowsy_start_time = None  # Reset if not drowsy

        # Check for key press to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        cv2.imshow("Drowsiness Detection", frame)

    webcamera.release()
    cv2.destroyAllWindows()

# Run the prediction function
predict_drowsiness()
